### 과제 url

url => https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1

과제: 무신사 코디맵 페이지(많은 댓글 수 기준 정렬)에 나와있는 60개의 코디맵에 대하여 각각 ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers'] 정보를 가져와서 csv 형태로 만든 다음, 해당 코드(.ipynb)와 csv 파일을 본인 github 과제 repository에 제출

Challenge:
1. 각 코디맵마다 클릭한 뒤 다시 이전페이지로 돌아오는 코드를 반복문에 추가하여 ['codimap_explain', 'codimap_hashtag', 'codimap_imgurl'] 정보도 가져오기
2.  page url을 format 함수를 이용해 수정한 뒤 반복문을 통해 1페이지부터 5페이지까지의 코디맵 정보 가져오기

# Challenge는 Optional Task로 필수적으로 해야하는 것은 아닙니다. 좀 더 심화된 크롤링을 원할경우 시도해보면 좋을 것 같아요!

In [ ]:
!pip3 install requests
!pip3 install bs4
!pip3 install selenium

In [1]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.10.0


Challenge:
1. 각 코디맵마다 클릭한 뒤 다시 이전페이지로 돌아오는 코드를 반복문에 추가하여 ['codimap_explain', 'codimap_hashtag', 'codimap_imgurl'] 정보도 가져오기

In [25]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 무신사 창 불러오기
url_1 = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url_1)
driver.implicitly_wait(2)
try:
    driver.maximize_window()
except:
    pass

interval = 1
prev_height = driver.execute_script("return document.body.scrollHeight")
while True:
        # 스크롤을 가장 아래로 내림
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        # 페이지 로딩 대기
        time.sleep(interval)

        # 현재 문서 높이를 가져와서 저장
        curr_height = driver.execute_script("return document.body.scrollHeight")
        if curr_height == prev_height:
            break

        prev_height = curr_height

In [26]:
columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers', 'codimap_explain', 'codimap_hashtag', 'codimap_imgurl']
values = []
soup = BeautifulSoup(driver.page_source, 'lxml')
data_rows = soup.find_all('li', attrs={'class':'style-list-item'})

for i, row in enumerate(data_rows):
    print('{}번째 코디맵 크롤링'.format(i+1))
    codimap = []
    
    category = row.find('span', attrs={'class':'style-list-information__text'})
    if category:
        category = category.get_text().strip()
        codimap.append(category)
    else:
        codimap.append('Something is wrong')
        print(f'{i+1}번째 카테고리 가져올때 문제발생')
        continue

    title = row.find('strong', attrs={'class':'style-list-information__title'})
    if title:
        title = title.get_text().strip()
        codimap.append(title)
    else:
        codimap.append('Something is wrong')
        print(f'{i+1}번째 타이틀 가져올때 문제발생')
        continue
    
    information = row.find_all('span', attrs={'class':'post-information__text'})

    date = information[0]                 
    if date:
        date = date.get_text().replace('""','').strip()
        codimap.append(date)
    else:
        codimap.append('Something is wrong')
        print(f'{i+1}번째 날짜 가져올때 문제발생')
        continue
    
    views = information[1]                 
    if views:
        views = views.get_text().replace('""','').strip()
        views = views[3:]        # 또는 views = re.sub(r'[^0-9]', '', views) re 사용
        codimap.append(views)
    else:
        codimap.append('Something is wrong')
        print(f'{i+1}번째 조회수 가져올때 문제발생')
        continue
    
    comments = information[2]           
    if comments:
        comments = comments.get_text().replace('""','').strip()
        comments = comments[3:]
        codimap.append(comments)
    else:
        codimap.append('Something is wrong')
        print(f'{i+1}번째 댓글수 가져올때 문제발생')
        continue
    
    # 1번째 사진 Xpath: /html/body/div[3]/div[3]/form/div[4]/div/ul/li[1]
    # 2번째 사진 Xpath: /html/body/div[3]/div[3]/form/div[4]/div/ul/li[2]

    codimap_button = driver.find_element(By.XPATH, '/html/body/div[3]/div[3]/form/div[4]/div/ul/li[{0}]'.format(i+1))
    codimap_button.click()
    
    soup2 = BeautifulSoup(driver.page_source, 'lxml')
    
    explain = soup2.find('p', attrs={'class':'styling_txt'})
    if explain:
        explain = explain.get_text().strip()
        codimap.append(explain)
    else:
        codimap.append('Something is wrong')
        print(f'{i+1}번째 카테고리 가져올때 문제발생')
        continue

    hashtag_lst = soup2.find_all('a', attrs={'class':'ui-tag-list__item'})
    if hashtag_lst:
        tag_lst = []
        for hashtag in (hashtag_lst):
            hashtag = '#' + hashtag['data-tagname']
            hashtag = hashtag.strip()
            tag_lst.append(hashtag)
        hashtags = ', '.join(map(str, tag_lst))
        codimap.append(hashtags)
        # print(hashtags)
    else:
        codimap.append('Something is wrong')
        print(f'{i+1}번째 카테고리 가져올때 문제발생')
    
    image_url = soup2.find('img', attrs={'class':'photo'})
    if image_url:
        image_url = image_url['src']
        if image_url.startswith('//'):
            image_url = 'https:' + image_url
        codimap.append(image_url)
    else:
        codimap.append('Something is wrong')
        print(f'{i+1}번째 카테고리 가져올때 문제발생')
        continue
        
    values.append(codimap)
    print(codimap)
    print('---------------------------------------------------')
    
    driver.back()


1번째 코디맵 크롤링
#톤온톤, #뉴트럴톤, #캠퍼스, #피크닉, #와이드핏, #오버핏, #꾸안꾸, #여름, #캐주얼, #벗딥, #엠엠엘지, #캐치볼, #예일, #반원 아틀리에, #토트백, #잡지/무크지, #캡/야구 모자, #캔버스/단화, #반소매 티셔츠
['캐주얼', '톤온톤 코디', '21.07.02', '16,041', '49', '편하게 입기 좋은 반팔 티셔츠와 와이드 팬츠를 연출하고 크로스 백으로 마무리한 캐주얼 룩', '#톤온톤, #뉴트럴톤, #캠퍼스, #피크닉, #와이드핏, #오버핏, #꾸안꾸, #여름, #캐주얼, #벗딥, #엠엠엘지, #캐치볼, #예일, #반원 아틀리에, #토트백, #잡지/무크지, #캡/야구 모자, #캔버스/단화, #반소매 티셔츠', 'https://image.msscdn.net/images/codimap/detail/4035/detail_4035_1_500.jpg?202307130107']
---------------------------------------------------
2번째 코디맵 크롤링
#봄, #캐주얼, #라퍼지스토어, #다이아몬드 레이라, #나이키, #엠프렌즈, #모티패스트, #이에스엔, #안경 소품, #디퓨저/캔들, #반소매 티셔츠, #셔츠/블라우스, #패션스니커즈화, #케이스/슬리브, #캡/야구 모자, #레더/라이더스 재킷, #맨투맨/스웨트셔츠
['캐주얼', '블랙 매니아', '21.02.23', '34,666', '46', '편안한 무드의 조거 팬츠에 블랙 스웨트셔츠를 매치한 후 볼캡으로 마무리한 캐주얼 룩', '#봄, #캐주얼, #라퍼지스토어, #다이아몬드 레이라, #나이키, #엠프렌즈, #모티패스트, #이에스엔, #안경 소품, #디퓨저/캔들, #반소매 티셔츠, #셔츠/블라우스, #패션스니커즈화, #케이스/슬리브, #캡/야구 모자, #레더/라이더스 재킷, #맨투맨/스웨트셔츠', 'https://image.msscdn.net/images/codimap/detail/1969/detail_1969_1_500.

#뉴트럴톤, #핀턱, #벌룬핏, #레이어드룩, #체크, #로고플레이, #퀼팅, #아메카지, #코듀로이, #겨울, #아메리칸 캐주얼, #프리즘웍스, #와일드 브릭스, #마인드브릿지, #코스트원더, #유니버셜 케미스트리, #레인보우샌들, #기타 아우터, #비니, #메신저/크로스 백, #잡지/무크지, #패딩 베스트, #셔츠/블라우스, #코튼 팬츠, #헌팅캡/베레모, #구두, #모카신/보트 슈즈
['아메리칸 캐주얼', '아메리칸 캐주얼', '21.12.21', '4,487', '10', '코듀로이 소재가 돋보이는 베스트와 셔츠를 레이어드하고 벌룬 핏 팬츠를 더해 마무리한 아메리칸 캐주얼 룩', '#뉴트럴톤, #핀턱, #벌룬핏, #레이어드룩, #체크, #로고플레이, #퀼팅, #아메카지, #코듀로이, #겨울, #아메리칸 캐주얼, #프리즘웍스, #와일드 브릭스, #마인드브릿지, #코스트원더, #유니버셜 케미스트리, #레인보우샌들, #기타 아우터, #비니, #메신저/크로스 백, #잡지/무크지, #패딩 베스트, #셔츠/블라우스, #코튼 팬츠, #헌팅캡/베레모, #구두, #모카신/보트 슈즈', 'https://image.msscdn.net/images/codimap/detail/10377/detail_10377_1_500.jpg?202307130107']
---------------------------------------------------
18번째 코디맵 크롤링
#편안한, #캠퍼스, #원마일웨어, #로고플레이, #여름, #캐주얼, #엠엠엘지, #제로, #캉골, #칼하트, #나이키, #캡/야구 모자, #트레이닝/조거 팬츠, #백팩, #반소매 티셔츠, #패션스니커즈화
['캐주얼', '편하게 완성', '21.07.21', '14,973', '10', '그린 컬러의 반팔 티셔츠와 스웨트 팬츠를 코디하고 스니커즈로 완성한 이지 캐주얼 룩', '#편안한, #캠퍼스, #원마일웨어, #로고플레이, #여름, #캐주얼, #엠엠엘지, #제로, #캉골, #칼하트, #나이키, #캡/야구 모자,

#모노톤, #단정한, #미니멀, #출근룩, #레더, #와이드핏, #가을, #포멀, #커스텀어클락, #시그니처, #슈커스, #탄틴, #베이직하우스, #수아레, #크리스챤모드, #반소매 티셔츠, #카디건, #슈트 팬츠/슬랙스, #안경 소품, #액자/포스터, #부츠, #브리프케이스, #긴소매 티셔츠, #쿼츠 아날로그
['포멀', '단정한 모습', '21.11.26', '5,645', '7', '미니멀한 실루엣의 롱 코트와 슬랙스를 매치하고 브리프케이스로 마무리한 포멀 룩', '#모노톤, #단정한, #미니멀, #출근룩, #레더, #와이드핏, #가을, #포멀, #커스텀어클락, #시그니처, #슈커스, #탄틴, #베이직하우스, #수아레, #크리스챤모드, #반소매 티셔츠, #카디건, #슈트 팬츠/슬랙스, #안경 소품, #액자/포스터, #부츠, #브리프케이스, #긴소매 티셔츠, #쿼츠 아날로그', 'https://image.msscdn.net/images/codimap/detail/9506/detail_9506_1_500.jpg?202307130107']
---------------------------------------------------
35번째 코디맵 크롤링
#아가일, #레더, #남친룩, #데이트, #슬림핏, #겨울, #댄디, #트릴리온, #필루미네이트, #쿠어, #커스텀에이드, #커스텀어클락, #247 서울, #엘엠씨, #토트백, #반소매 티셔츠, #셔츠/블라우스, #데님 팬츠, #안경 소품, #부츠, #겨울 더블 코트, #머플러, #카디건
['댄디', '훈훈한 스타일', '21.12.29', '3,507', '7', '아가일 패턴이 돋보이는 카디건과 데님 팬츠를 매치하고 머플러를 더해 연출한 댄디 룩', '#아가일, #레더, #남친룩, #데이트, #슬림핏, #겨울, #댄디, #트릴리온, #필루미네이트, #쿠어, #커스텀에이드, #커스텀어클락, #247 서울, #엘엠씨, #토트백, #반소매 티셔츠, #셔츠/블라우스, #데님 팬츠, #안경 소품, #부츠, #겨울 더

#봄, #캐주얼, #유니폼브릿지, #커버낫, #캡/야구 모자, #에코백, #코튼 팬츠, #캔버스/단화, #맨투맨/스웨트셔츠
['캐주얼', '캐주얼 룩의 정석', '21.03.12', '10,970', '5', '스트라이프 패턴이 돋보이는 카라 티셔츠와 치노 팬츠를 매치하고 오렌지 컬러의 캠프캡으로 포인트 준 캐주얼 룩', '#봄, #캐주얼, #유니폼브릿지, #커버낫, #캡/야구 모자, #에코백, #코튼 팬츠, #캔버스/단화, #맨투맨/스웨트셔츠', 'https://image.msscdn.net/images/codimap/detail/2237/detail_2237_1_500.jpg?202307130107']
---------------------------------------------------
52번째 코디맵 크롤링
#봄, #스트릿, #로파이, #베이어다이나믹, #플레어업, #어나더유스, #앤더슨벨, #레더/라이더스 재킷, #기타 시계, #캡/야구 모자, #헤드폰, #트레이닝/조거 팬츠, #목걸이/펜던트
['스트릿', '스트라이프 활용법', '21.02.19', '2,793', '5', '데미지 디테일이 돋보이는 스트라이프 니트에 카고 팬츠를 매치하고 하이 탑 스니커즈로 마무리한 스트릿 룩', '#봄, #스트릿, #로파이, #베이어다이나믹, #플레어업, #어나더유스, #앤더슨벨, #레더/라이더스 재킷, #기타 시계, #캡/야구 모자, #헤드폰, #트레이닝/조거 팬츠, #목걸이/펜던트', 'https://image.msscdn.net/images/codimap/detail/1937/detail_1937_1_500.jpg?202307130107']
---------------------------------------------------
53번째 코디맵 크롤링
#그래픽, #체크, #레이어드룩, #플리스, #키높이, #글로시, #귀여운, #레더, #여친룩, #짧치, #겨울, #걸리시, #아파트먼트, #타게토, #어피스오브케이크, #룩캐스트, #닥터마틴, #이로

In [27]:
df = pd.DataFrame(values, columns=columns)
display(df)

,codimap_category,codimap_title,codimap_date,views,comment_numbers,codimap_explain,codimap_hashtag,codimap_imgurl
0,캐주얼,톤온톤 코디,21.07.02,"16,041",49,편하게 입기 좋은 반팔 티셔츠와 와이드 팬츠를 연출하고 크로스 백으로 마무리한 캐주얼 룩,"#톤온톤, #뉴트럴톤, #캠퍼스, #피크닉, #와이드핏, #오버핏, #꾸안꾸, #여...",https://image.msscdn.net/images/codimap/detail...
1,캐주얼,블랙 매니아,21.02.23,"34,666",46,편안한 무드의 조거 팬츠에 블랙 스웨트셔츠를 매치한 후 볼캡으로 마무리한 캐주얼 룩,"#봄, #캐주얼, #라퍼지스토어, #다이아몬드 레이라, #나이키, #엠프렌즈, #모...",https://image.msscdn.net/images/codimap/detail...
2,캐주얼,오늘의 코디,22.03.22,"12,194",38,로고 디테일이 돋보이는 스웨트셔츠와 데님 팬츠를 코디하고 스니커즈로 완성한 캐주얼 룩,"#로고플레이, #오버핏, #스티치, #캠퍼스, #봄, #캐주얼, #유니폼브릿지, #...",https://image.msscdn.net/images/codimap/detail...
3,댄디,추천 댄디 룩,21.09.15,"20,180",27,담백한 디테일의 니트와 셔츠를 레이어드하고 데님 팬츠를 더해 완성한 댄디 룩,"#데이트, #그래픽, #레이어드룩, #가을, #댄디, #엘무드, #뮤지엄바이비컨, ...",https://image.msscdn.net/images/codimap/detail...
4,캐주얼,릴랙스 룩,21.02.23,"23,532",22,플라워 자수가 눈에 띄는 니트에 조거 팬츠를 더한 후 레드 볼캡으로 컬러 포인트를 ...,"#봄, #캐주얼, #카시오, #코닥, #제로, #와릿이즌, #뉴발란스, #커버낫, ...",https://image.msscdn.net/images/codimap/detail...
5,스트릿,힙하다 힙해,21.02.03,"35,282",21,레더 블루종 점퍼에 오버사이즈 니트와 조거 팬츠를 매치하고 블루 컬러 볼캡으로 멋을...,"#봄, #스트릿, #제로, #월간, #라퍼지스토어, #키치오브제, #블론드나인, #...",https://image.msscdn.net/images/codimap/detail...
6,캐주얼,심플한 코디,21.07.30,"39,570",18,로고 프린팅이 돋보이는 반팔 티셔츠와 스웨트 팬츠를 매치하고 스니커즈로 완성한 이지...,"#로고플레이, #편안한, #원마일웨어, #와이드핏, #여름, #캐주얼, #리, #세...",https://image.msscdn.net/images/codimap/detail...
7,댄디,데이트 코디,21.12.24,"10,305",18,깔끔한 무드의 니트에 데님 팬츠를 스타일링하고 스니커즈로 마무리한 댄디 룩,"#남친룩, #레더, #데이트, #테이퍼드핏, #오버핏, #소개팅, #스티치, #겨울...",https://image.msscdn.net/images/codimap/detail...
8,댄디,훈훈 남친 룩,21.03.24,"19,339",16,체크 패턴이 돋보이는 블레이저에 셔츠와 데님 팬츠를 더하고 담백한 구두로 마무리한 ...,"#봄, #댄디, #모디파이드, #반, #락포트, #수아레, #슈트/블레이저 재킷, ...",https://image.msscdn.net/images/codimap/detail...
9,캐주얼,지금 입기 좋아,21.08.31,"20,208",16,오버 핏 스웨트셔츠와 베이지 컬러 팬츠를 코디하고 더비 슈즈로 완성한 캐주얼 룩,"#꾸안꾸, #남친룩, #핀턱, #캠퍼스, #와이드핏, #여름, #캐주얼, #브래디백...",https://image.msscdn.net/images/codimap/detail...


In [29]:
df.to_csv('김채연_크롤링과제_Challenge1.csv', encoding = 'utf-8-sig')


Challenge : 


2.  page url을 format 함수를 이용해 수정한 뒤 반복문을 통해 1페이지부터 5페이지까지의 코디맵 정보 가져오기

In [33]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

codimap_final = [] # 최종 1~5페이지 정보 담을 리스트
page_dict = {0:'1페이지', 1:'2페이지', 2:'3페이지', 3:'4페이지', 4:'5페이지'}

for p in range(5):
    url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page={0}'.format(p+1)
    driver.get(url)
    driver.implicitly_wait(2)
    try:
        driver.maximize_window()
    except:
        pass
    
    interval = 1
    prev_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 스크롤을 가장 아래로 내림
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        # 페이지 로딩 대기
        time.sleep(interval)

        # 현재 문서 높이를 가져와서 저장
        curr_height = driver.execute_script("return document.body.scrollHeight")
        if curr_height == prev_height:
            break

        prev_height = curr_height
    
    columns = ['page', 'codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers', 'codimap_explain', 'codimap_hashtag', 'codimap_imgurl']
    values = []
    soup = BeautifulSoup(driver.page_source, 'lxml')
    data_rows = soup.find_all('li', attrs={'class':'style-list-item'})
    
    print('{0}번째 페이지 크롤링 시작'.format(p+1))
    print('---------------------------------------------------')
    for i, row in enumerate(data_rows):
        print('{}번째 코디맵 크롤링'.format(i+1))
        codimap = []
        codimap.append(page_dict[p])
        
        category = row.find('span', attrs={'class':'style-list-information__text'})
        if category:
            category = category.get_text().strip()
            codimap.append(category)
        else:
            codimap.append('Something is wrong')
            print(f'{i+1}번째 카테고리 가져올때 문제발생')
            continue

        title = row.find('strong', attrs={'class':'style-list-information__title'})
        if title:
            title = title.get_text().strip()
            codimap.append(title)
        else:
            codimap.append('Something is wrong')
            print(f'{i+1}번째 타이틀 가져올때 문제발생')
            continue

        information = row.find_all('span', attrs={'class':'post-information__text'})

        date = information[0]                 
        if date:
            date = date.get_text().replace('""','').strip()
            codimap.append(date)
        else:
            codimap.append('Something is wrong')
            print(f'{i+1}번째 날짜 가져올때 문제발생')
            continue

        views = information[1]                 
        if views:
            views = views.get_text().replace('""','').strip()
            views = views[3:]        # 또는 views = re.sub(r'[^0-9]', '', views) re 사용
            codimap.append(views)
        else:
            codimap.append('Something is wrong')
            print(f'{i+1}번째 조회수 가져올때 문제발생')
            continue

        comments = information[2]           
        if comments:
            comments = comments.get_text().replace('""','').strip()
            comments = comments[3:]
            codimap.append(comments)
        else:
            codimap.append('Something is wrong')
            print(f'{i+1}번째 댓글수 가져올때 문제발생')
            continue

        # 1번째 사진 Xpath: /html/body/div[3]/div[3]/form/div[4]/div/ul/li[1]
        # 2번째 사진 Xpath: /html/body/div[3]/div[3]/form/div[4]/div/ul/li[2]

        codimap_button = driver.find_element(By.XPATH, '/html/body/div[3]/div[3]/form/div[4]/div/ul/li[{0}]'.format(i+1))
        codimap_button.click()

        soup2 = BeautifulSoup(driver.page_source, 'lxml')

        explain = soup2.find('p', attrs={'class':'styling_txt'})
        if explain:
            explain = explain.get_text().strip()
            codimap.append(explain)
        else:
            codimap.append('Something is wrong')
            print(f'{i+1}번째 카테고리 가져올때 문제발생')
            continue

        hashtag_lst = soup2.find_all('a', attrs={'class':'ui-tag-list__item'})
        if hashtag_lst:
            tag_lst = []
            for hashtag in (hashtag_lst):
                hashtag = '#' + hashtag['data-tagname']
                hashtag = hashtag.strip()
                tag_lst.append(hashtag)
            hashtags = ', '.join(map(str, tag_lst))
            codimap.append(hashtags)
            # print(hashtags)
        else:
            codimap.append('Something is wrong')
            print(f'{i+1}번째 카테고리 가져올때 문제발생')

        image_url = soup2.find('img', attrs={'class':'photo'})
        if image_url:
            image_url = image_url['src']
            if image_url.startswith('//'):
                image_url = 'https:' + image_url
            codimap.append(image_url)
        else:
            codimap.append('Something is wrong')
            print(f'{i+1}번째 카테고리 가져올때 문제발생')
            continue
            
        driver.back()
        values.append(codimap)
        print('---------------------------------------------------')
    

    df = pd.DataFrame(values, columns = columns)
    codimap_final.append(df)


1번째 페이지 크롤링 시작
---------------------------------------------------
1번째 코디맵 크롤링
---------------------------------------------------
2번째 코디맵 크롤링
---------------------------------------------------
3번째 코디맵 크롤링
---------------------------------------------------
4번째 코디맵 크롤링
---------------------------------------------------
5번째 코디맵 크롤링
---------------------------------------------------
6번째 코디맵 크롤링
---------------------------------------------------
7번째 코디맵 크롤링
---------------------------------------------------
8번째 코디맵 크롤링
---------------------------------------------------
9번째 코디맵 크롤링
---------------------------------------------------
10번째 코디맵 크롤링
---------------------------------------------------
11번째 코디맵 크롤링
---------------------------------------------------
12번째 코디맵 크롤링
---------------------------------------------------
13번째 코디맵 크롤링
---------------------------------------------------
14번째 코디맵 크롤링
---------------------------------------------------
15번째 코디맵 크롤링
-------------------

---------------------------------------------------
6번째 코디맵 크롤링
---------------------------------------------------
7번째 코디맵 크롤링
---------------------------------------------------
8번째 코디맵 크롤링
---------------------------------------------------
9번째 코디맵 크롤링
---------------------------------------------------
10번째 코디맵 크롤링
---------------------------------------------------
11번째 코디맵 크롤링
---------------------------------------------------
12번째 코디맵 크롤링
---------------------------------------------------
13번째 코디맵 크롤링
---------------------------------------------------
14번째 코디맵 크롤링
---------------------------------------------------
15번째 코디맵 크롤링
---------------------------------------------------
16번째 코디맵 크롤링
---------------------------------------------------
17번째 코디맵 크롤링
---------------------------------------------------
18번째 코디맵 크롤링
---------------------------------------------------
19번째 코디맵 크롤링
---------------------------------------------------
20번째 코디맵 크롤링
-----------------------------

---------------------------------------------------
11번째 코디맵 크롤링
---------------------------------------------------
12번째 코디맵 크롤링
---------------------------------------------------
13번째 코디맵 크롤링
---------------------------------------------------
14번째 코디맵 크롤링
---------------------------------------------------
15번째 코디맵 크롤링
---------------------------------------------------
16번째 코디맵 크롤링
---------------------------------------------------
17번째 코디맵 크롤링
---------------------------------------------------
18번째 코디맵 크롤링
---------------------------------------------------
19번째 코디맵 크롤링
---------------------------------------------------
20번째 코디맵 크롤링
---------------------------------------------------
21번째 코디맵 크롤링
---------------------------------------------------
22번째 코디맵 크롤링
---------------------------------------------------
23번째 코디맵 크롤링
---------------------------------------------------
24번째 코디맵 크롤링
---------------------------------------------------
25번째 코디맵 크롤링
-------------------------

In [34]:
df_final = pd.concat(codimap_final).reset_index(drop=True)
display(df_final)

,page,codimap_category,codimap_title,codimap_date,views,comment_numbers,codimap_explain,codimap_hashtag,codimap_imgurl
0,1페이지,캐주얼,톤온톤 코디,21.07.02,"16,046",49,편하게 입기 좋은 반팔 티셔츠와 와이드 팬츠를 연출하고 크로스 백으로 마무리한 캐주얼 룩,"#톤온톤, #뉴트럴톤, #캠퍼스, #피크닉, #와이드핏, #오버핏, #꾸안꾸, #여...",https://image.msscdn.net/images/codimap/detail...
1,1페이지,캐주얼,블랙 매니아,21.02.23,"34,668",46,편안한 무드의 조거 팬츠에 블랙 스웨트셔츠를 매치한 후 볼캡으로 마무리한 캐주얼 룩,"#봄, #캐주얼, #라퍼지스토어, #다이아몬드 레이라, #나이키, #엠프렌즈, #모...",https://image.msscdn.net/images/codimap/detail...
2,1페이지,캐주얼,오늘의 코디,22.03.22,"12,196",38,로고 디테일이 돋보이는 스웨트셔츠와 데님 팬츠를 코디하고 스니커즈로 완성한 캐주얼 룩,"#로고플레이, #오버핏, #스티치, #캠퍼스, #봄, #캐주얼, #유니폼브릿지, #...",https://image.msscdn.net/images/codimap/detail...
3,1페이지,댄디,추천 댄디 룩,21.09.15,"20,182",27,담백한 디테일의 니트와 셔츠를 레이어드하고 데님 팬츠를 더해 완성한 댄디 룩,"#데이트, #그래픽, #레이어드룩, #가을, #댄디, #엘무드, #뮤지엄바이비컨, ...",https://image.msscdn.net/images/codimap/detail...
4,1페이지,캐주얼,릴랙스 룩,21.02.23,"23,534",22,플라워 자수가 눈에 띄는 니트에 조거 팬츠를 더한 후 레드 볼캡으로 컬러 포인트를 ...,"#봄, #캐주얼, #카시오, #코닥, #제로, #와릿이즌, #뉴발란스, #커버낫, ...",https://image.msscdn.net/images/codimap/detail...
...,...,...,...,...,...,...,...,...,...
295,5페이지,스트릿,타이다이 코디,21.05.26,"1,369",3,타이다이 패턴이 돋보이는 반팔 티셔츠에 스트링 팬츠를 코디하고 크로스 백으로 완성한...,"#타이다이, #유니크, #봄, #스트릿, #논논, #살로몬, #디퍼런트이즈.오케이,...",https://image.msscdn.net/images/codimap/detail...
296,5페이지,캐주얼,따라 입어봐,22.10.19,"5,260",3,레터링이 돋보이는 후드 티셔츠와 데님 팬츠를 코디하고 스니커즈로 마무리한 캐주얼 룩,"#레터링, #와이드핏, #오버핏, #꾸안꾸, #캠퍼스, #리버서블, #가을, #캐주...",https://image.msscdn.net/images/codimap/detail...
297,5페이지,캐주얼,손이 가는 조합,22.09.19,"6,060",3,블루 컬러가 매력적인 스웨트셔츠와 스타디움 재킷을 코디하고 데님 팬츠로 마무리한 캐...,"#레터링, #꾸안꾸, #컬러블록, #콜라보, #가을, #캐주얼, #트래블, #언다이...",https://image.msscdn.net/images/codimap/detail...
298,5페이지,스트릿,힙하게 입어봐,22.10.11,"3,778",3,자수 디테일이 돋보이는 스타디움 재킷과 데님 팬츠를 매치하고 스니커즈로 멋을 더한 ...,"#모노톤, #유니크, #힙합, #심볼, #그래픽, #벨루어, #자수, #컬러블록, ...",https://image.msscdn.net/images/codimap/detail...


In [35]:
df_final.to_csv('김채연_크롤링과제_Challenge2.csv', encoding = 'utf-8-sig')